<a href="https://colab.research.google.com/github/rono516/DSciencE/blob/master/ExploratoryDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!pip install --upgrade plotly

     |████████████████████████████████| 27.7 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [6]:
#create a directory called kaggle
!mkdir ~/.kaggle
#copy kaggle.json file into kaggle
!cp kaggle.json ~/.kaggle/
#change mode of kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
#download dataset
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
#unzip our dataset
!unzip 2021-olympics-in-tokyo.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 55.0MB/s]
Archive:  2021-olympics-in-tokyo.zip
  inflating: Athletes.xlsx           
  inflating: Coaches.xlsx            
  inflating: EntriesGender.xlsx      
  inflating: Medals.xlsx             
  inflating: Teams.xlsx              


In [4]:
#install pyspark tool
!pip install pyspark

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 62.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1f135ce5632fb8ebc17662a2abc5ff213d702296fee1b435954a31187dcb9c1f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [5]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [8]:
#Convert the excel files to csv
def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep=',' , index =False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

teams.head(5)

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [15]:
#Sorting using Rank By Total
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [25]:
#Library for pandas dataframe |>
import plotly.express as px
fig = px.bar(medals, x='Team/NOC', y =['Gold', 'Silver', 'Bronze'], color_discrete_sequence=['Gold', 'Silver', 'Brown'], title='Number of Medals per country' )
fig.show()


In [36]:
coaches.head(2)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN


In [41]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [65]:
#order by NOC orders by alphabets of country
#select from Coaches table
#toPandas converts to pandas
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from Coaches group by NOC order by NOC').toPandas()
#before changing to spark use pd_coaches_query.show(7) to show values
pd_coaches_query.head(7)

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [55]:
fig = px.bar(pd_coaches_query, x = 'NOC', y ='Count', color_discrete_sequence=['Black'], title='Number of coaches from each country')
fig.show()

In [63]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('Select * from Athletes').show(5)
#order by country name and discipline
# spark.sql('select * from athletes order by NOC, Discipline').show()

+-----------------+------+-------------------+
|             Name|   NOC|         Discipline|
+-----------------+------+-------------------+
|  AALERUD Katrine|Norway|       Cycling Road|
|      ABAD Nestor| Spain|Artistic Gymnastics|
|ABAGNALE Giovanni| Italy|             Rowing|
|   ABALDE Alberto| Spain|         Basketball|
|    ABALDE Tamara| Spain|         Basketball|
+-----------------+------+-------------------+
only showing top 5 rows



In [67]:
fig = px.bar(pd_coaches_query, x='NOC', y='Count', color_discrete_sequence=['Red'], title='Number of coaches by country')
fig.show()

In [79]:
first_athletes = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline, Count(*) as Count from athletes group by
              NOC, Discipline order by NOC, Discipline''').toPandas()
first_athletes.head(10)

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
1,Afghanistan,Shooting,1
2,Afghanistan,Swimming,1
3,Afghanistan,Taekwondo,1
4,Albania,Artistic Gymnastics,1
5,Albania,Athletics,1
6,Albania,Judo,1
7,Albania,Shooting,1
8,Albania,Swimming,2
9,Albania,Weightlifting,2


In [80]:
fig = px.bar(first_athletes, x='NOC', y='Count',color='Discipline' , title= "First Athletes for every country per discipline")
fig.show()

In [81]:
fig.show()

In [95]:
# athletes['Discipline'].value_counts().head()
importantsport = ['Athletics', 'Swimming', 'Football']
first_athletes2 = first_athletes.loc[first_athletes['Discipline'].isin(importantsport)]
# first_athletes2
fig = px.bar(first_athletes2, x="NOC", y= "Count", color="Discipline", title="Plotting important sports")
fig.show()

In [127]:
# gender.head(5)
fig2 = px.bar(gender, x ='Discipline', y= ['Female', 'Male'],color_discrete_sequence=['Green', 'Blue'], title= 'Gender entries' )
fig2.show()

In [110]:
#plotting using plotly
#gender dataframe has 46 rows
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'}, {'type':'domain'}]]*23
fig3 = make_subplots(rows=23, cols=2,
                     subplot_titles=gender['Discipline'], specs=specs)

sub_figs = []
for index, row in gender.iterrows():

  sub_fig = go.Pie(labels=['Female','Male'], values=[row['Female'],
                                                     row['Male']])
  sub_figs.append(sub_fig)
k = 0
for i in range(1, 24):
  for j in range(1, 3):
    fig3.add_trace(sub_figs[k], i, j)
    k += 1

fig3.update_layout(showlegend=False, height=10000, width=800, title_text="Distribution of gender amongst each game")
fig3.update_traces(textposition='inside', textinfo='label+percent', hoverinfo="label+value+percent")

fig3 = go.Figure(fig3)
#show pie chart
# fig3.show()

In [111]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [119]:
spark.read.csv('Teams.csv', header=True, inferSchema=True,).createOrReplaceTempView('teams')


In [129]:
pd_teams = spark.sql('select NOC as NOC, Discipline as Discipline, Count(*) as Count from teams group by NOC, Discipline order by NOC, Discipline').toPandas()
pd_teams.head()

fig4 = px.bar(pd_teams, x='NOC', y='Count', color='Discipline', title="Teams per country")
fig4.show()
